In [1]:
import pandas as pd
import json
from Conversation import Conversation
import Text_Preprocessing as TP
import math
from operator import itemgetter
msdialog_filepath = "data/MSDialog/Intent/MSDialog-Intent.json"
msdialog_classes = 

In [49]:
with open(msdialog_filepath) as f:
    data = json.load(f)
    

corpus = []
questions = []
follow_ups = []
for j, thread in enumerate(data.keys()):
    messages = data[thread]["utterances"]
    for i, message in enumerate(messages):
        if i == 0 and j > 20 and j < 30:
            q_id = message["id"]
            questions.append(message)
        elif j > 20 and j < 30:
            message["q_id"] = q_id
            follow_ups.append(message)
        corpus.append(message)

In [50]:
words_occurence = {}
for question in corpus:
    question_cleaned = TP.preprocess_text(question["utterance"])
    for word in set(question_cleaned):
        if word not in words_occurence:
            words_occurence[word] = 0
        words_occurence[word] += 1
print("{} unique words".format(len(words_occurence)))
del words_occurence['']

16436 unique words


In [51]:
min_occurences = 5
document_count = len(corpus)
idf = {key:math.log2(document_count / val) for key, val in words_occurence.items() if val > min_occurences}
print("{} words with more than {} occurences".format(len(idf), min_occurences))

2738 words with more than 5 occurences


In [52]:
for question in questions:
    tf = {}
    cleaned_message = TP.preprocess_text(question["utterance"])
    for word in cleaned_message:
        if word not in tf and word in idf:
            tf[word] = 0
        if word in idf:
            tf[word] += 1
    tf_idf = {}
    for word in tf.keys():
        tf_idf[word] = tf[word] * idf[word]

    tf_idf = dict(sorted(tf_idf.items(), key = itemgetter(1), reverse = True)[:5])
    question["tfidf"] = tf_idf
    
for follow_up in follow_ups:
    tf = {}
    cleaned_message = TP.preprocess_text(follow_up["utterance"])
    for word in cleaned_message:
        if word not in tf and word in idf:
            tf[word] = 0
        if word in idf:
            tf[word] += 1
    tf_idf = {}
    for word in tf.keys():
        tf_idf[word] = tf[word] * idf[word]

    tf_idf = dict(sorted(tf_idf.items(), key = itemgetter(1), reverse = True)[:5])
    follow_up["tfidf"] = tf_idf

In [53]:
for follow_up in follow_ups:
    tf_idf_message = follow_up["tfidf"]
    best_match = ("", 0)
    for question in questions:
        tf_idf_question = question["tfidf"]
        intersect = set(tf_idf_question.keys()).intersection(set(tf_idf_message.keys()))
        if len(intersect) > 0:
            score = sum([tf_idf_question[item] for item in list(intersect)])
            if score > best_match[1]:
                best_match = (question["id"], score)
    follow_up["match"] = best_match[0]

In [54]:
not_matched = 0
matched_correctly = 0
match_error = 0
for follow_up in follow_ups:
    if follow_up["match"] == "":
        not_matched += 1
    elif follow_up["q_id"] != follow_up["match"]:
        print(follow_up["id"], follow_up["q_id"], follow_up["match"])
        match_error += 1
    else:
        matched_correctly += 1
print("{} Correctly Matched. {} Wrongly Matched. {} Not Matched".format(matched_correctly, match_error, not_matched))

3363 3361 3014
4512 4511 3014
4513 4511 3361
7 Correctly Matched. 3 Wrongly Matched. 14 Not Matched


## MS Dialog


In [2]:
import csv
with open(msdialog_filepath) as f:
    data = json.load(f)


with open('file.csv', 'rb') as f:
    reader = csv.reader(f)
    your_list = list(reader)

In [9]:
for j, thread in enumerate(data.keys()):
    messages = data[thread]["utterances"]
    for i, message in enumerate(messages):
        print(thread, i+1)

16 1
16 2
16 3
25 1
25 2
25 3
25 4
25 5
36 1
36 2
36 3
36 4
36 5
40 1
40 2
40 3
40 4
40 5
40 6
74 1
74 2
74 3
74 4
74 5
74 6
75 1
75 2
75 3
77 1
77 2
77 3
77 4
96 1
96 2
96 3
96 4
96 5
96 6
96 7
103 1
103 2
103 3
103 4
103 5
103 6
103 7
103 8
103 9
103 10
106 1
106 2
106 3
147 1
147 2
147 3
147 4
147 5
